In [4]:
from langchain.agents import create_agent
from langchain_ollama import ChatOllama
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
def get_weather(city:str) -> str:
    """Get weather for a given city."""
    return f"The weather in {city} is sunny with a high of 75°F."

def pick_toolmessage(result):
    messages=result["messages"]
    return [m for m in messages if isinstance(m,ToolMessage)][-1]


agent = create_agent(
    model=ChatOllama(model="deepseek-v3.1:671b-cloud"),
    tools=[
        get_weather
    ],
    system_prompt="You are a helpful assistant"
)
chain=agent | RunnableLambda(pick_toolmessage) | StrOutputParser()
response=chain.invoke({"messages": [{"role": "user", "content": "What's the weather like in New York?"}]})
response


'The weather in New York is sunny with a high of 75°F.'

# build a real-world agent

Define system prompt

In [5]:
system_prompt = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

create tools

In [6]:

from dataclasses import dataclass
from langgraph.runtime import get_runtime
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"The weather in {city} is sunny with a high of 75°F."

@dataclass
class Context:
    user_id: str

def get_user_location() -> str:
    """Retrieve user information based on user ID."""
    runtime=get_runtime(Context)
    user_id=runtime.context.user_id
    return "Florida" if user_id=="1" else "SF"

config model

In [7]:
from langchain_ollama import ChatOllama
model=ChatOllama(model="deepseek-v3.1:671b-cloud")

define response format

In [8]:
from dataclasses import dataclass
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    punny_response:str
    weather_conditions:str|None=None

add memory

In [9]:
from langgraph.checkpoint.memory import InMemorySaver
checkpointer=InMemorySaver()

create and run the agent

In [10]:
agent=create_agent(
    model=model,
    system_prompt=system_prompt,
    tools=[
        get_weather_for_location,
        get_user_location
    ],
    context_schema=Context,
    response_format=ResponseFormat,
    checkpointer=checkpointer
)
config={"configurable":{"thread_id":"1"}}
response=agent.invoke({"messages": [{"role": "user", "content": "What's the weather like?"}]},config=config,context=Context(user_id="1"))
print(response['structured_response'])
response=agent.invoke({"messages": [{"role": "user", "content": "thank you!"}]},config=config,context=Context(user_id="1"))
print(response['structured_response'])

ResponseError: upstream error (status code: 502)